In [ ]:
import numpy as np
import cv2 as cv
import json
from keras.models import model_from_json




# Method for image processing

def process_image(frame):    
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray,(5,5),2)
    adaptive_thresh = cv.adaptiveThreshold(blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY_INV,11,2)
    ret, final = cv.threshold(adaptive_thresh, 70, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    
    return final


######################################################################################################

# Pre-processing 


# Load model from JSON file 

f = open ('model.json', "r")
data = json.loads(f.read())
data = json.dumps(data)
model = model_from_json(data)


#Initialize dictionary 

word_dict = {0:'',1:'a', 2:'b', 3:'c', 4:'d', 5:'e', 6:'f', 7:'g', 8:'h', 9:'i', 10:'j', 
             11:'k', 12:'l', 13:'m', 14:'n', 15:'o', 16:'p', 17:'q', 18:'r', 19:'s', 20:'t',
             21:'u', 22:'v', 23:'w', 24:'x', 25:'y', 26:'z'} 


######################################################################################################


# Processing the video frame by frame

capture = cv.VideoCapture(0)

while True:
    
    # capture each frame 
    isTrue, frame = capture.read()
    frame = cv.rectangle(frame,(0,0),(250,400),(0,255,0),thickness = 2)

    cv.imshow('Video',frame)
        
    # select and perform operations on ROI
    frame2 = frame[0:400,0:250]
    frame2 = process_image(frame2)
    
    cv.imshow('Modified',frame2)
    
    # resize the modified ROI 
    frame2 = cv.resize(frame2,(128,128),interpolation = cv.INTER_AREA)
    frame2 = np.reshape(frame2,(1,128,128,1))
    
    # Predict the result using model
    pred = model.predict(frame2)

#     cv.putText(frame, word_dict[np.argmax(pred)], (400,400), cv.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    
    # Print our predicted result
    print(word_dict[np.argmax(pred)])      
    
    # Press 'd' to quit
    if cv.waitKey(20) & 0xFF==ord('d'):
        break

        
        
########################################################################################################
        
# Post processing 
    
f.close()
capture.release()
cv.destroyAllWindows()


# TODO-------------------------------------------
# flip the image before passing it to model
# background setup check
# GUI buildup for interaction and output display
#-----------------------------------------------